##### Import libraries

In [1]:
import numpy as np
import pandas as pd

import nltk
from nltk.corpus import stopwords
import string
import nltk
nltk.download('stopwords') ##downloading stopwords to remove from text

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\AASHISH\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

##### Reading cvs file

In [46]:
df = pd.read_csv('yelp.csv')
df.head()

,business_id,date,review_id,stars,text,type,user_id,cool,useful,funny
0,9yKzy9PApeiPPOUJEtnvkg,2011-01-26,fWKvX83p0-ka4JS3dc6E5A,5,My wife took me here on my birthday for breakf...,review,rLtl8ZkDX5vH5nAx9C3q5Q,2,5,0
1,ZRJwVLyzEJq1VAihDhYiow,2011-07-27,IjZ33sJrzXqU-0X6U8NwyA,5,I have no idea why some people give bad review...,review,0a2KyEL0d3Yb1V6aivbIuQ,0,0,0
2,6oRAC4uyJCsJl1X0WZpVSA,2012-06-14,IESLBzqUCLdSzSqm0eCSxQ,4,love the gyro plate. Rice is so good and I als...,review,0hT2KtfLiobPvh6cDC8JQg,0,1,0
3,_1QQZuf4zZOyFCvXc0o6Vg,2010-05-27,G-WvGaISbqqaMHlNnByodA,5,"Rosie, Dakota, and I LOVE Chaparral Dog Park!!...",review,uZetl9T0NcROGOyFfughhg,1,2,0
4,6ozycU1RpktNG2-1BroVtw,2012-01-05,1uJFq2r5QfJG_6ExMRCaGw,5,General Manager Scott Petello is a good egg!!!...,review,vYmM4KTsC8ZfQBg-j5MWkw,0,0,0


In [3]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10000 entries, 0 to 9999
Data columns (total 10 columns):
business_id    10000 non-null object
date           10000 non-null object
review_id      10000 non-null object
stars          10000 non-null int64
text           10000 non-null object
type           10000 non-null object
user_id        10000 non-null object
cool           10000 non-null int64
useful         10000 non-null int64
funny          10000 non-null int64
dtypes: int64(4), object(6)
memory usage: 781.3+ KB


##### There is no nan values in dataframe

In [4]:
df.isna().sum()

business_id    0
date           0
review_id      0
stars          0
text           0
type           0
user_id        0
cool           0
useful         0
funny          0
dtype: int64

In [5]:
x = df['text'].values
y = df['stars'].values

In [6]:
print(x.shape)
print(y.shape)

(10000,)
(10000,)


In [7]:
def analyzer_process(text):
   
    # Check characters to see if they are in punctuation
    text_punctuation = [char for char in text if char not in string.punctuation]

    # Join the characters again to form the string.
    text_punctuation = ''.join(text_punctuation)
    
    # Now just remove any stopwords
    return [word for word in text_punctuation.split() if word.lower() not in stopwords.words('english')]

In [8]:
valid_revs =[]
valid_labels = []

for review in range(x.shape[0]):
    if (y[review] == 5 or y[review]==1):
        valid_revs.append(x[review])
        valid_labels.append(y[review])

out_revs = np.array(valid_revs)
print (out_revs.shape)

out_labels = np.array(valid_labels)
print (out_labels.shape)


(4086,)
(4086,)


In [9]:
from sklearn.feature_extraction.text import CountVectorizer

In [10]:
count_vect = CountVectorizer(analyzer=analyzer_process).fit(out_revs)

In [11]:
X  = count_vect.transform(out_revs)

In [12]:
from sklearn.model_selection import train_test_split

x_train, x_test, y_train, y_test = train_test_split(X, out_labels)

In [13]:
print (x_train.shape)
print (x_test.shape)
print (y_train.shape)
print (y_test.shape)

(3064, 26435)
(1022, 26435)
(3064,)
(1022,)


##### Logistic Regression

In [14]:
from sklearn.linear_model import LogisticRegression

In [15]:
clf = LogisticRegression(C=5)

In [16]:
clf.fit(x_train,y_train)

LogisticRegression(C=5, class_weight=None, dual=False, fit_intercept=True,
          intercept_scaling=1, max_iter=100, multi_class='ovr', n_jobs=1,
          penalty='l2', random_state=None, solver='liblinear', tol=0.0001,
          verbose=0, warm_start=False)

In [17]:
y_pred = clf.predict(x_test)

In [18]:
print (y_pred)

[5 5 5 ... 5 5 1]


In [19]:
from sklearn.metrics import accuracy_score
accuracy_score(y_test, y_pred)

0.9197651663405088

In [20]:
from sklearn.feature_extraction.text import TfidfTransformer
tf_transformer = TfidfTransformer(use_idf=False).fit(x_train)
x_train_tf = tf_transformer.transform(x_train)
x_test_tf = tf_transformer.transform(x_test)

In [21]:
clf2 = LogisticRegression(C=5)

In [22]:
clf2.fit(x_train_tf.toarray(),y_train)
y_pred = clf2.predict(x_test_tf.toarray())
accuracy_score(y_test, y_pred)

0.9266144814090019

In [23]:
from sklearn.feature_extraction.text import TfidfTransformer
tf_transformer = TfidfTransformer(use_idf=True).fit(x_train)
x_train_tfidf = tf_transformer.transform(x_train)
x_test_tfidf = tf_transformer.transform(x_test)
clf2 = LogisticRegression(C=5)
clf2.fit(x_train_tfidf.toarray(),y_train)
y_pred = clf2.predict(x_test_tfidf.toarray())
accuracy_score(y_test, y_pred)

0.9178082191780822

##### KNN

In [24]:
from sklearn.neighbors import KNeighborsClassifier

knn = KNeighborsClassifier(n_neighbors=10)

In [25]:
knn.fit(x_train,y_train)

KNeighborsClassifier(algorithm='auto', leaf_size=30, metric='minkowski',
           metric_params=None, n_jobs=1, n_neighbors=10, p=2,
           weights='uniform')

In [26]:
y_pred = knn.predict(x_test)

In [27]:
print (y_pred)

[5 5 5 ... 5 5 5]


In [28]:
from sklearn.metrics import accuracy_score
accuracy_score(y_test, y_pred)

0.8346379647749511

##### Decision tree

In [29]:
from sklearn.tree import DecisionTreeClassifier
dt = DecisionTreeClassifier()

dt.fit(x_train,y_train)
y_pred = dt.predict(x_test)

In [30]:
accuracy_score(y_test, y_pred)

0.8639921722113503

##### Random Forest

In [31]:
from sklearn.ensemble import RandomForestClassifier
rmfr = RandomForestClassifier()
rmfr.fit(x_train,y_train)
y_pred = rmfr.predict(x_test)

C:\Users\AASHISH\Anaconda3\lib\site-packages\sklearn\ensemble\weight_boosting.py:29: DeprecationWarning: numpy.core.umath_tests is an internal NumPy module and should not be imported. It will be removed in a future NumPy release.
  from numpy.core.umath_tests import inner1d


In [32]:
accuracy_score(y_test, y_pred)

0.8864970645792564

##### Multinomial Naive Bayes

In [33]:
from sklearn.naive_bayes import MultinomialNB
mnb = MultinomialNB()
mnb.fit(x_train,y_train)
y_pred = mnb.predict(x_test)

In [34]:
accuracy_score(y_test, y_pred)

0.9080234833659491

In [35]:
from sklearn.metrics import confusion_matrix, classification_report

print(confusion_matrix(y_test, y_pred))
print('\n')
print(classification_report(y_test, y_pred))

[[106  67]
 [ 27 822]]


             precision    recall  f1-score   support

          1       0.80      0.61      0.69       173
          5       0.92      0.97      0.95       849

avg / total       0.90      0.91      0.90      1022



##### SVM

In [36]:
from sklearn.svm import SVC
svm = SVC()
svm.fit(x_train,y_train)
y_pred = svm.predict(x_test)

In [37]:
accuracy_score(y_test, y_pred)

0.8307240704500979

In [42]:
pr = df['text'][0]
print(pr)
print("Actual Rating: ",df['stars'][0])
pr_t = count_vect.transform([pr])
print("Predicted Rating:", clf2.predict(pr_t)[0])


My wife took me here on my birthday for breakfast and it was excellent.  The weather was perfect which made sitting outside overlooking their grounds an absolute pleasure.  Our waitress was excellent and our food arrived quickly on the semi-busy Saturday morning.  It looked like the place fills up pretty quickly so the earlier you get here the better.

Do yourself a favor and get their Bloody Mary.  It was phenomenal and simply the best I've ever had.  I'm pretty sure they only use ingredients from their garden and blend them fresh when you order it.  It was amazing.

While EVERYTHING on the menu looks excellent, I had the white truffle scrambled eggs vegetable skillet and it was tasty and delicious.  It came with 2 pieces of their griddled bread with was amazing and it absolutely made the meal complete.  It was the best "toast" I've ever had.

Anyway, I can't wait to go back!
Actual Rating:  5
Predicted Rating: 5


In [43]:
ar = df['text'][16]
print(ar)
print("Actual Rating: ",df['stars'][16])
ar_t = count_vect.transform([ar])
print("Predicted Rating:",mnb.predict(ar_t)[0])

We went here on a Saturday afternoon and this place was incredibly empty.  They had brunch specials going on, including $2 bloody mary's and mimosas, but we were more in the mood for lunch.  Except for the bloody mary, I had to try one.  It came out in a high-ball-sized glass.  Boo!  But it was really tasty. Yay!  The hubby remembered a sign outside the restaurant a few weeks back that said they had Arrogant Bastard, and he got a 22 oz bottle for $4.75.  Hey, that's not fair!!

Next up: the wings.  We were a bit hesitant to order them when the waitress informed us that they are "seasoned" but not sauced, so they can't be ordered hot.  We did ask for them crispy though, and the waitress even asked the cooks to throw them back in for a few minutes when they came out not visibly crispy.  These non-traditional wings were actually pretty damn good.  The seasoning was a little spicy and salty with just a hint of sweet.  If I were in the mood for the tang and kick of Frank's Hot Sauce, these 

In [45]:
nr = df['text'][23]
print(nr)
print("Actual Rating: ",df['stars'][23])
nr_t = count_vect.transform([nr])
print("Predicted Rating:",clf2.predict(nr_t)[0])


U can go there n check the car out. If u wanna buy 1 there? That's wrong move! If u even want a car service from there? U made a biggest mistake of ur life!! I had 1 time asked my girlfriend to take my car there for an oil service, guess what? They ripped my girlfriend off by lying how bad my car is now. If without fixing the problem. Might bring some serious accident. Then she did what they said. 4 brand new tires, timing belt, 4 new brake pads. U know why's the worst? All of those above I had just changed 2 months before!!! What a trashy dealer is that? People, better off go somewhere!
Actual Rating:  1
Predicted Rating: 1
